# Transferlearning

In [1]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from my_class import my_class as my
%matplotlib inline

In [2]:
# Set directories
data_path_clear = "./../data/data/clear/billund/2016/"
data_path_foggy = "./../data/data/foggy/billund/2017/"

In [3]:
pic_path_clear = my.list_pics(data_path_clear)
pic_path_foggy = my.list_pics(data_path_foggy)
pic_path_clear = [pic_path_clear[ii] for ii in range(len(pic_path_clear)) if ".jpg" in pic_path_clear[ii]]
pic_path_foggy = [pic_path_foggy[ii] for ii in range(len(pic_path_foggy)) if ".jpg" in pic_path_foggy[ii]]


#n_sample = 10
#pic_path_clear = np.random.choice(pic_path_clear, size=n_sample, replace=False)
#pic_path_foggy = np.random.choice(pic_path_foggy, size=n_sample, replace=False)

pic_path = np.concatenate((pic_path_clear, pic_path_foggy))
# pic_path = [pic_path[ii] for ii in range(len(pic_path)) if ".jpg" in pic_path[ii]]

n_clear = len(pic_path_clear)
n_foggy = len(pic_path_foggy)
n = len(pic_path)

## Create target variable and feature matrix

In [4]:
Y_clear = np.zeros(n_clear, dtype=int)
Y_foggy = np.ones(n_foggy, dtype=int)
Y = np.concatenate((Y_clear, Y_foggy), axis=0)
# balance(Y)
# one hot
b = np.zeros((len(Y), len(set(Y))))
b[np.arange(len(Y)), Y] = 1
Y = b
n_classes = Y.shape[1]
classes = ["clear", "foggy"]
np.save("./../data/tmp/Y_transfer.npy", Y)

In [5]:
ratio = 1
channels = 3
update = True
#
# if update:
if False:
    pics = my.img_to_nparr(pic_path=pic_path, 
                           img_height = 288, 
                           img_width = 384, 
                           rat = ratio,
                           ch = channels,
                           verbose = False)

    # only consider the 3 /5 top of the picture...    
    #pics = pics[:, 0:int(pics.shape[1] / 5 * 4),:,:]
    # dimensions picture
    image_height, image_width, _ = pics[1].shape
    n_pixels = image_height * image_width

## Feature extraction

In [6]:
features = ["Dark channel", "sobel_VARsob", "sobel_TEN", 
            "laplace_sum", "laplace_var", "pct_overexposed"]
n_features = len(features)

In [7]:
# if update:
if False:
    X = np.zeros((n, n_features))
    for ii in range(n):
        print(str(ii + 1) + " of " + str(n), end="\r")
        feature_list = []
        # dark channel
        dc = my.get_dark_channel(pics[ii], win=20)
        # close to 1 -> presents of fog
        feature_list.append(np.mean(dc / 255.0)) 

        # sobel edge filtering
        S = my.sobel_filter(pics[ii]),
        feature_list.append(my.VARsob(S))
        feature_list.append(my.TEN(S) / n_pixels)
        
        # laplace
        L = my.lapalce_filter(pics[ii])
        feature_list.append(np.sum(abs(L)) / n_pixels)
        feature_list.append(np.var(abs(L)) / n_pixels)
        
        # pct. overexposed pixels
        feature_list.append(my.overexposed_pixels(pics[ii]) / n_pixels)
        
        # add to design matrix
        X[ii,:] = feature_list
    # if updated save new... 
    print("Updated...")
    np.save("./../data/tmp/X_transfer.npy", X)
else:
    X = np.load("./../data/tmp/X_transfer.npy")
    

In [8]:
def performance(pred, Y):
    """
    asd
    """
    from sklearn.metrics import confusion_matrix
    import numpy as np

    def array_to_latex(tbl):
        for ii in range(tbl.shape[0]):
            tmp_str = ''
            for jj in range(tbl.shape[1]):
                if jj != 0:
                    tmp_str += ' & ' + "{:.0f}".format(tbl[ii,jj])  
                else:
                    tmp_str += "{:.0f}".format(tbl[ii,jj]) 

            tmp_str += ' \\\\ '
            print(tmp_str)

    def performance_measure(pred_test, Y_test):
        #
        cm = confusion_matrix(y_pred = pred_test,
            y_true = Y_test, 
            labels = list(range(len(set(Y_test)))))
        TP = np.diag(cm)
        FP = np.sum(cm, axis=0) - np.diag(cm)
        FN = np.sum(cm,axis=1) - np.diag(cm)
        TN = np.sum(cm) - (FP+FN+TP)
        #
        precision = TP/ (TP + FP)
        recall = TP / (TP + FN)
        F1 = np.multiply(2, np.multiply(precision, recall) / np.add(precision, recall))
        acc = (TP+TN)/(TP+FP+FN+TN)
        #
        return TP, FP, precision, recall, F1, acc, cm


    TP, FP, precision, recall, F1, Acc, cm = performance_measure(pred_test=pred, Y_test=np.argmax(Y, axis=1))
    print('--------------------------------------------')
    print('Average for all classes')
    print('Accurcy:   %f' %(np.mean(Acc)))
    print('Precision: %f' %(np.mean(precision)))
    print('Recall:    %f' %(np.mean(recall)))
    print('F1:        %f' %(np.mean(F1)))

    #
    print("std.\n")
    array_to_latex(cm)
    # 
    print("\npct.\n")
    cm_norm = cm / cm.astype(np.float).sum(axis=1, keepdims=True) * 100
    array_to_latex(cm_norm)

    print("\n\nPaste into latex..\n\n")
    tmp = np.ndarray((2,6))
    tmp[0:2,0:2] = cm_norm
    
    tmp[0:2,2] = precision * 100
    tmp[0:2,3] = recall * 100
    tmp[0:2,4] = F1 * 100
    tmp[0:2,5] = Acc * 100
    #
    array_to_latex(tmp)

## Model assessment
### Balance

In [9]:
import collections as col
tmp = list(dict(col.Counter(np.argmax(Y,1))).values())

tmp / np.sum(tmp)

array([ 0.9197815,  0.0802185])

### RF

In [12]:
Y

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [10]:
# load model
from sklearn.externals import joblib
clf = joblib.load("./../data/tmp/clf.pkl")
# make predictions
pred = clf.predict(X)
performance(pred, Y)

--------------------------------------------
Average for all classes
Accurcy:   0.905726
Precision: 0.641259
Recall:    0.585244
F1:        0.603461
std.

14491 & 495 \\ 
1041 & 266 \\ 

pct.

97 & 3 \\ 
80 & 20 \\ 


Paste into latex..


97 & 3 & 93 & 97 & 95 & 91 \\ 
80 & 20 & 35 & 20 & 26 & 91 \\ 


### KNN

In [11]:
# load model
neigh = joblib.load("./../data/tmp/neigh.pkl")
# make predictions
pred = neigh.predict(X)
performance(pred, Y)

--------------------------------------------
Average for all classes
Accurcy:   0.102375
Precision: 0.539614
Recall:    0.511695
F1:        0.099352
std.

362 & 14624 \\ 
1 & 1306 \\ 

pct.

2 & 98 \\ 
0 & 100 \\ 


Paste into latex..


2 & 98 & 100 & 2 & 5 & 10 \\ 
0 & 100 & 8 & 100 & 15 & 10 \\ 
